<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item">
    
<li><span><a href="#1.-Introduction" data-toc-modified-id=" Introduction"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li>
    
<li><span><a href="#2.-Environment-of-the-project" data-toc-modified-id="2.Environment of the project"><span class="toc-item-num">2&nbsp;&nbsp;</span>Environment of the project</a></span></li>
    
<li><span><a href="#3.-Method-overview" data-toc-modified-id="3.-Method-overview"><span class="toc-item-num">3&nbsp;&nbsp;</span>Method overview</a></span></li>

<li><span><a href="#4.-Computer-vision" data-toc-modified-id="4.-Computer-vision"><span class="toc-item-num">4&nbsp;&nbsp;</span>Computer vision</a></span>
    <ul class="toc-item"><li><span><a href="#4.1.-Pre-processing-the-raw-image:" data-toc-modified-id="4.1.-Pre-processing-the-raw-image:"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Pre-processing the raw image:</a></span></li>
    <li><span><a href="#4.2.-Identification-of-Robot-State:" data-toc-modified-id="4.2.-Identification-of-Robot-State:"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Identification of Robot State:</a></span></li>
    <li><span><a href="#4.3.-Grid-Analysis:" data-toc-modified-id="4.3.-Grid-Analysis:"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Grid Analysis:</a></span></li></ul></li>
    
<li><span><a href="#5.-Path-finding" data-toc-modified-id="#5.-Path-finding"><span class="toc-item-num">5&nbsp;&nbsp;</span>Path finding</a></span></li>
    
<li><span><a href="#6.-Local-Navigation" data-toc-modified-id="#6.-Local-Navigation"><span class="toc-item-num">6&nbsp;&nbsp;</span>Local Navigation</a></span></li>
    
<li><span><a href="#7.-Filtering" data-toc-modified-id="#7.-Filtering"><span class="toc-item-num">7&nbsp;&nbsp;</span>Filtering</a></span></li>
    
<li><span><a href="#8.-Controller" data-toc-modified-id="#8.-Controller"><span class="toc-item-num">8&nbsp;&nbsp;</span>Controller</a></span></li>
    
 <li><span><a href="#9.-Project-run" data-toc-modified-id="#9.-Project-run"><span class="toc-item-num">9&nbsp;&nbsp;</span>Project run</a></span></li>

<li><span><a href="#10.-Conclusion" data-toc-modified-id="#10.-Conclusion"><span class="toc-item-num">10&nbsp;&nbsp;</span>Conclusion</a></span></li>

In [1]:
from IPython.display import Image, display

# Vision guided navigation with Thymio - Final project
Authors: Ahmed Boubakry, Charles de Fournas, Julie Favre, Paul Richard

## 1. Introduction

The objective of this project is to develop a comprehensive robotic system for the Thymio robot, enabling it to navigate efficiently on a two-dimensional plane. The primary focus involves implementing advanced algorithms and methods to achieve global and local navigation, obstacle avoidance, accurate position estimation, and trajectory correction. The key components of our project include:

- Computer Vision in order to create the map and retrieve the robot's position: Utilizing computer vision techniques, we aim to create a map of the environment and accurately determine the Thymio robot's position in real-time<br><br>


- A* algorithm to find the optimal path:,Implementing the A* algorithm to find the optimal path from the starting position to the specified goal, considering the map and any obstacles encountered<br><br>

- Local navigation to avoid physical objects: Developing local navigation methods to enable the Thymio robot to dynamically adjust its path in real-time, effectively avoiding physical obstacles encountered during its movement<br><br>


- Filtering with Extended Kalman Filter to estimate accuretly the position: Employing the Extended Kalman Filter for precise and accurate estimation of the robot's position, enhancing the overall reliability of the navigation system and still providing the position of the robot if we don't see the camera<br><br>

- Controller to correct the trajectory after every move: The robot will change his direction in accord with the camera and theorical position.

## 2. Environment of the project

We have opted for a black and white grid as our environment, where white cells signify walkable areas and black cells represent walls. The robot's objective is to determine the optimal path from its starting position to a designated goal cell, indicated in blue.

For visualization purposes, we print the grid on an A0 page with 8 columns and 6 rows. The page's borders serve as reference points for the camera to deduce the grid. We've outlined the grid using black tape, aiding in its detection.

Obstacles are depicted by black cartoon representations, and the goal is visually distinguished by a blue cartoon. Additionally, on the robot, we've strategically positioned two black points – a smaller one in front of a larger one – to help the camera ascertain the robot's orientation.

The camera is mounted on a tripod with an extension, positioning it vertically above the center of our environment.

To enhance the camera's visibility, we've strategically placed multiple lamps to illuminate the area. These lamps are intended to provide ample lighting for improved camera performance.

In consideration of the Thymio sensor, obstacles are designed to be solid, ensuring effective detection by the sensor. It's important to note that these obstacles are shaped differently from rounds, as our camera primarily detects round forms for our robot.

<div style="text-align:center">
    <img src="image/env_2.jpg" alt="environnement" width="400"/>
</div>


## 3. Method overview

The program begins with the initialization of global variables and the importation of modules. Initially, we extract the grid and the robot position. The subsequent significant part of our code focuses on finding the optimal path to reach the destination.

Once the setup is complete, the main loop initiates. The robot moves from one position to another, and at regular intervals, the program analyzes and corrects its position. During each iteration, the loop checks for potential obstacles. If an obstacle is detected, a corresponding function is triggered to navigate around it. Otherwise, the program proceeds as usual, initiating a new loop to progress to the next step.

This loop continues until the final destination is reached. The structure ensures that the robot dynamically adapts its path, corrects its trajectory, and navigates around obstacles, providing a robust and efficient navigation system.

<img src="image/logic_map.jpg" alt="Method overview" width="800"/>

## 4. Computer vision

The integration of computer vision is crutial in our project for initializing and periodically updating the data points necessary for autonomous navigation. This encompasses the robot's current location, the orientation, the goal's coordinates, and the matrices that detail the state and the centers of the grid cells. To fulfill these requirements, we employ the info_robot_cam() function as follows:

```python
def info_robot_cam():
    global center_matrix, state_matrix
    image_not_good = True
    while image_not_good: 
        image_not_good = False
        ret, frame = cam.read()
        if ret:
            result_image = frame.copy()  # Initialiser result_image avec une copie de l'image actuelle
            filtered_image = pre_processing(result_image)
            
            # Blob detector parameters for circle detection
            params = cv2.SimpleBlobDetector_Params()
            
            # Create a blob detector with the configured parameters
            detector = cv2.SimpleBlobDetector_create(params)
            
            # Detect blobs in the image
            keypoints = detector.detect(filtered_image)
            
            # Update the info about the robot
            robot_state, d = robot_info(keypoints, result_image,params)
    
            state_matrix, center_matrix, x_end, y_end =grid_setting(filtered_image, result_image)
            
            state_matrix = np.array(state_matrix)
            data = symetrie_lignes(state_matrix)
            for i in range(num_cases_y):
                for j in range(num_cases_x):
                    if (state_matrix[i][j] == None) or d>100: #and (x_end ==0 or y_end == 0)
                        image_not_good = True
        else:
            print('no image found')
            image_not_good = False
            
    return data, center_matrix, x_end, y_end, robot_state, result_image
``````

### 4.1. Pre-processing the raw image:

   
Our process commences with the acquisition of a live camera feed, upon which we conduct a series of filtration and computer vision operations, when calling '*pre_processing(image)*'. Initially, we convert the live feed into a grayscale image to standardize the input. Subsequently, we apply a bilateral filter to preserve edges while diminishing noise. Lastly, we enhance the contrast through Contrast Limited Adaptive Histogram Equalization (CLAHE), which equips us with a fully detailed grayscale image. 

<div style="display: flex; justify-content: space-around; align-items: center; flex-wrap: wrap;">
    <div style="flex-basis: 50%; text-align: center;">
        <img src="image/raw_image.png" alt="Step 1" style="max-width: 90%;"/>
        <p><strong>Step 1: Raw Image</strong></p>
    </div>
    <div style="flex-basis: 50%; text-align: center;">
        <img src="image/image_grey.png" alt="Step 2" style="max-width: 90%;"/>
        <p><strong>Step 2: Grey Image</strong></p>
    </div>
    <div style="flex-basis: 50%; text-align: center;">
        <img src="image/bilateral_filtered.png" alt="Step 3" style="max-width: 90%;"/>
        <p><strong>Step 3: Bilateral Filtered</strong></p>
    </div>
    <div style="flex-basis: 50%; text-align: center;">
        <img src="image/clahe_applied.png" alt="Step 4" style="max-width: 90%;"/>
        <p><strong>Step 4: CLAHE Applie</strong></p>
    </div>
</div>


***Images at the different step of the pre-processing***

### 4.2. Identification of Robot State:

Using the *robot_info(keypoints, result_image, params)* function, we analyze the image to identify key markers on the robot, specifically two distinct circles. The largest circle, positioned at the robot's center, determines its current coordinates. The relative positioning of the two circles enables us to infer the robot's orientation by comparing the vector formed by these circles against the horizontal axis.


<div style="text-align: center;"> 
    <img src="image/image_with_circles_drawn.png" alt="environnement" width="500"/> 
    <p><strong>Figure 1:</strong> Robot with localisation circles</p> 
</div>

### 4.3. Grid Analysis:
 
We employ the grid_setting(filtered_image) function to define or update the state and center matrices. By recognizing the largest contour, which outlines our operational grid, and by locating its four corners, we ascertain the pixel positions of each cell's center, thus updating our center_matrix. Subsequently, by examining the intensity of the centers, we differentiate between occupied and vacant cells, updating our state_matrix accordingly.

<div style="text-align: center;"> 
    <img src="image/image_with_red_points.png" alt="environnement" width="500"/> 
    <p><strong>Figure 2:</strong> Map with the red points in the center of the cells</p> 
</div>

As our map is in a large format: A0, since we are taking the photo from above the map, there is a slight distortion on the sides which means that when the robot is on a square, its center does not match the center below it. See '*Figure 4*'. That's why we have calibrated a matrix that puts all the points in the right places. We placed the robot in the center of a square for each column and compared the value of the robot's center in x with the theoretical center: '*Figure 3*'

<div style="text-align: center;"> 
    <img src="image/image_with_corrected_red_points.png" alt="environnement" width="500"/> 
    <p><strong>Figure 3:</strong> image_with_red_points</p> 
</div>

<div style="text-align: center;"> 
    <img src="image/schema_vision.jpg" alt="environnement" width="300"/> 
    <p><strong>Figure 4:</strong> Diagram of Image Correction</p> 
</div>

## 5. Path finding

This module is very dependant on the vision module, since the vision module maps the area. The vision module returns a state matrix, giving us the information on which square is occupied or not. After manipulating this matrix so it can work conviniently with the function that runs with the A* function, we can run the algorithm. The shortest path is then given to us in the form of a double array, called in the code `path`.

We have chosen the A* algorithm because our sceario, which is a robot trying to find the goal in a 'maze', corresponds exactly on how we want the shortest path be computed from. Using the heuristic function h wasn't really necessary since all the distances the robot runs have an equal distance from each other. But in the case where our robot can use 8 directions paths, it would've helped in fiding the shortest path.

Once `path` is filled, we run a loop that will make the robot move to each location one by one, starting by loading the values of its current position and the next one : 

            for i in range(len(path)-1):
                x1, y1 = path[i]
                x2, y2 = path[i + 1]


We make the robot move three times to move from a location to another, so it has the time between each movement to check the obstacles, and update the `obstacle_state` variable. If the robot encounters an obstacle, it will run another loop that makes the robot avoid the obstacle (see Local Navigation section).

<div style="text-align: center;"> 
    <img src="image/image_with_corrected_red_points.png" alt="environnement" width="500"/> 
    <p><strong>Figure 5:</strong> Path planning execution </p> 
</div>

All the movements that the robot makes are computed via mathematical functions, (namely, `calculate_angle_degrees` and `calculate_distance`), and given to `move_robot` and `rotate_robot`, who will give a command to a motor and put to sleep using `time.sleep()` during the time the command need to be applied.

Then, we check if the robot has reached its goal by looking at the robot position and the goal position via the camera. If the two positions matches (with a margin error around 50 pixels), then we stop the program.


## 6. Local Navigation

### Theory : 

To ensure obstacle avoidance, we have opted for a convinient way to do so by using the Artificial Neural Network method. By using the five sensors in front of the Thymio, the robot can modify the speed of its wheel depending on the value returned by the thymio. We can use the following equation to compute the speed sent to the motors :

\begin{array}{rcl}
v_{Left} = w_{1l}x_1 + w_{2l}x_2 + w_{3l}x_3 + w_{4l}x_4 + w_{5l}x_5 \\
v_{Right} = w_{1r}x_1 + w_{2r}x_2 + w_{3r}x_3 + w_{4r}x_4 + w_{5r}x_5
\end{array}

Thanks to the ANN method, ach proximity sensor has one weight coefficient. The coefficients $x_1, \cdots, x_5$ are used as the measurements of the horizontal distances that the robot can sense.

#### Obstacle detection using ANN : 

    speed0 = 50      
    SpeedGainObst = [6, 4, -2, -6, -8]
    SpeedLeftWheel = speed0
    SpeedRightWheel = speed0

    for i in range(5):
        speedLeftWheel += obst[i] * SpeedGainObst[i] // 100
        speedRightWheel += obst[i] * SpeedGainObst[4 - i] // 100

    return speedLeftWheel, speedRightWheel

### How to find the weights? 

In our code, we used the following weights : $[w_{1l}, w_{2l}, w_{3l}, w_{4l}, w_{5l}]=[0.06,0.04,-0.02,-0.06,-0.08]$ and $[w_{1r}, w_{2r}, w_{3r}, w_{4r}, w_{5r}]=[-0.08,-0.06,-0.02,0.04,0.06]$, by using a nominal speed of 100, and all of this by testing and correcting our values.  

These coefficients are used so that the obstacle is dodged in a reasonnable manner (we want to stay in the velocity bound, and not give an angle which is too tight or too wide).

### Switching from local navigation to global navigation : 

In order to be able to return in the main state after making the global navigation, we use a function that checks the robot state (state_robot). When all the sensor do not detect something in front of them, and has been detecting obstacles before, the state becomes 2. When the case is the same but the sensor has never been detecting something before, the state remains 1. And finally, when the sensor detects and obstacle, we enter in the "obstacle avoidance" mode, which is described by the state 0.

### Kidnapping 

The objective of the kidnapping prevention mechanism for the Thymio robot is to detect and respond to instances where the robot is lifted off the ground, signaling a potential kidnapping scenario. This is achieved through continuous monitoring of the ground sensor, which detects changes in the robot's contact with the ground surface.

The Thymio robot is equipped with a ground sensor that provides real-time information about the contact between the robot and the ground. The detection mechanism relies on regular checks of the ground sensor readings during the robot's movement. The code continuously monitors the sensor output to ensure that the robot is in contact with the ground. If a deviation from the expected sensor values occurs, indicating that the robot is no longer on the ground, the system interprets this as a potential kidnapping event.

Upon detecting that the robot is not on the ground, the code initiates a predefined response strategy. The chosen approach involves resetting the robot's state and restarting the code from the beginning. We use the variable `was_kidnapped` to ensure that we can distinguish the case where the robot needs to restart or not.

## 7. Filtering

To be sure that we have the correct position of the robot even when the camera fails, we implemented a filtering feature. As our model was not linear, we chose the Extended Kalman Filter. This filter takes as input the position measured by the camera, and the instaneous speed of the robot. Every time it is called, it updates the position based on the previous position, the camera input, and the model of the robot (where it is supposed to be given the speed, the time elapsed, and the previous position).

We call the filter after every movement, giving the time period and the speed, thanks to the method `filter_call()` below.

The rest of the code is all in ``ExtendedKalmanFilter.py``

The key elements of the EKF are:

#### State Space Model

The system is modeled using a state-space representation. The state at time step $k$ is denoted by $x_k$, and the system dynamics are represented by two matrices: $A$ (representing the system if the robot is not moving) and $B$ representing the system if the robot is moving with speed $u_k$. B is defined by a method B(orientation, dt).
The model in movement is then computed by:

$x_k = A \cdot (x_{k-1}) + (B(orientation,dt)) \cdot (u_{k-1}) + (v_{k-1})$

Here, $(u_{k-1})$ is the control input, (in our case, the linear speed and the angular speed) and $v_{k-1}$ is the process noise. We chose $(v_{k-1} = [TODO])$ for our model not to have too much weight.

#### Measurement Model

Measurements $z_k$ are related to the state $x_k$ through a measurement function $h$, with added measurement noise $v_k$:

$ z_k = h(x_k) + v_k $. Here, our measurements are the camera inputs for the position in x,y of the robot (in pixels on the map) and its orientation (in degrees, with 0 being the right side of the map, and positive in the trigonometric direction). Our measurement noise matrix is $v_k = [TODO]$. 

We also have a matrix $R$, the measurement noise covariance matrix, and the matrix $R_{no cam}$, which is used instead of R when the camera gives no input.
We chose this way in order to have the measurements weight a lot in the computations, but then way less when the camera is blocked, and being able to estimated correctly the position.
#### Prediction Step

In the prediction step, the EKF estimates the new state based on the previous state estimate and the system dynamics. Here is the predict step:

$x_k$ computed as explained above
$P_k = A_{k-1} \cdot P_{k-1} \cdot A_{k-1}^T + (Q_k)$

Where:
- $P_k^-$ is the predicted state covariance matrix.
- $Q_{k-1}$ is the process noise covariance matrix.

#### Update Step

In the update step, the predicted state is corrected based on the measurements:

$y_k = z_k - ((H_k \cdot x_k) + (w_k))$

$S_k = H_k \cdot P_k \cdot H_k^T + R_k$ (choosing the correct R matrix according to the camera input)

$K_k = P_k \cdot H_k^T \cdot pinv(S_k)$

$x_k = (x_k + (K_k \cdot y_k))$


Where:
- \(K_k\) is the Kalman Gain.
- np.linalg.pinv is the method to compute the Moore-Penrose pseudo inverse of a matrix

We then update the new $P_k$ and $x_k$ to be ready for the next step.
Our estimated position is the last x_k computed!  

We use this position in the corrector, for our robot to have good control even without the measurements from the camera.


```python 
def filter_call(left, right, dt):
    """Makes the call to the filter using the initialised ekf
        Args:
        right: right speed of the motor in thymio unit (same unit as we give the motor)
        left: left speed
        Returns: estimated positions x, y, angle in pixels, degrees
        """
    global ekf
    #dt = update_time()
    speed = get_speed(right, left)
    _, _, _, _, position, _ = info_robot_cam()
    cam_pos = np.array([position.x, position.y, position.angle])
    return ekf.filter(cam_pos, speed, dt)
``````

## 8. Controller

As our robot advance and follow the path calculated by the global navigation, our robot sometimes deviate. We decided that a correction fonction is needed to assure the global trajectory. The first idea was to create a PID to follow the measured angle. But we couldn't apply the PID to our algoritm. That's why we create a second version to control specificly the direction correction.

After every cell the robot moved to, we check if its in the expected orientation. If the difference $d\theta$ between its orientation and the expected one is larger than 5 degrees, we correct it:

- if $d\theta < 5$: we do not modify it
- if $5<d\theta < 15$: we orientate it to have the expected orientation 
- if $d\theta > 15$: we orientate it to aim on the next cell's center

Moreover, if the robot just had to avoid an obstacle using local navigation, we orientate it in the direction of the second next cell; like this, it wont try to go in the celle the obstacle is still in. 

We call the controller everytime the robot made it safely to a cell, everytime we (re)compute A* for it to start with orientation 0, and just after we avoided an obstacle. 

## 9. Project run

main loop !
blablater sur les choix de synchrone vs asynchrone et quelles methodes on appelle quand


## 10. Conclusion

In conclusion, this project highlighted the challenges of collaborative programming, where differing perspectives and interpretations can lead to difficulties. When bugs arose in the main algorithm, identifying and resolving them proved challenging due to the lack of comprehension of the algorithm in its entierity.


Our experience with GitHub proved invaluable for project synchronization and streamlining algorithm transmission among team members. Moving forward, we recognize the importance of defining variables at the project's outset to avoid errors during the merge process. Additionally, establishing a standardized unit for distance, normalized during camera initialization, will enhance consistency.







### 10. References

ChatGPT to help with debugging

filter: https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/
